# Chapter 2: GPS Navigation

GPS Navigation is a whole universe of knowledge, and there is a lot to learn. 
Here you have an exercise that will help you to get used to the movement of the car and navigating with GPS.<br>

<p style="background:#EE9023;color:white;">Exercise U2.1</p>

The objective of this exercise is to make the car move through GPS readings to a defined location, or WayPoint.<br>
To guide you, follow these steps:<br>
<ul>
<li>Create a GPS Subscriber: Subscribe to the **/fix** topic. Then, extract the latitude and longitude. Also, make a function that calculates the distance from the current position to a given waypoint. This distance has to be determined through an algorithm for Earth distance calculations. For instance, through the Vincency method.
https://en.wikipedia.org/wiki/Vincenty%27s_formulae
</li>
<li>Create a Class that publishes in the appropriate topic to move the car. In this case, move without any security system, which is publishing in the **/catvehicle/cmd_vel_safe.**</li>
<li>Create an Action Server that, given a waypoint, returns the distance to that waypoint until the target is reached. Because the distance is calculated in meters, it will be enough with integer values.</li>
<li>Finally, create an Action Client that calls this server and moves the car to the given waypoint. You will have to implement an algorithm that, as distances change, can change the car's direction to get as close as possible.</li>
</ul>

<p style="color: green">**NOTE**: Bear in mind that there is already a security feature implemented in the car. If the Twist messages aren't published at rates greater than 5Hz in the **/catvehicle/cmd_vel_safe** topic, it stops until the next message is sent.<br>
This is due to the **"DeadMansSwitch,"** which we will talk about in the next chapter.</p>

Just to make it clear, try the following:

In [ ]:
rostopic pub /catvehicle/cmd_vel_safe geometry_msgs/Twist "linear:               
  x: 1.0                      
  y: 0.0                      
  z: 0.0                      
angular:                      
  x: 0.0                      
  y: 0.0                      
  z: 0.0"

This should only move the car for 1/5 of a second, which is the period corresponding to 5Hz of the DeadMansSwitch system. To make it move, you will have to publish at a rate of 5Hz or higher.

In [ ]:
rostopic pub -r5 /cvehicle/cmd_vel_safe geometry_msgs/Twist "linear:           
  x: 1.0                      
  y: 0.0                      
  z: 0.0                      
angular:                      
  x: 0.0                      
  y: 0.0                      
  z: 0.0"

And when you **CTRL-C**, it stops. If there is a system failure, this keeps the car from going wild. This way, if there is any mulfunction, the car will simply stop moving.

<p style="background:#EE9023;color:white;">END Exercise U2.1</p>

<p style="background:green;color:white;">Solution Exercise U2.1</p>

Please try to do it by yourself, unless you get stuck or need some inspiration. You will learn much more if you fight for each exercise.

<img src="img/robotignite_logo_text.png"/>

Here you have a possible solution to the core aspects of the exercise:

### Create a GPS Subscriber

In [1]:
#! /usr/bin/env python

import rospy
from sensor_msgs.msg import NavSatFix
#https://pypi.python.org/pypi/geopy
from geopy.distance import vincenty

"""
uint8 COVARIANCE_TYPE_UNKNOWN=0                                                                                                 
uint8 COVARIANCE_TYPE_APPROXIMATED=1                                                                                            
uint8 COVARIANCE_TYPE_DIAGONAL_KNOWN=2                                                                                          
uint8 COVARIANCE_TYPE_KNOWN=3                                                                                                   
std_msgs/Header header                                                                                                          
  uint32 seq                                                                                                                    
  time stamp                                                                                                                    
  string frame_id                                                                                                               
sensor_msgs/NavSatStatus status                                                                                                 
  int8 STATUS_NO_FIX=-1                                                                                                         
  int8 STATUS_FIX=0                                                                                                             
  int8 STATUS_SBAS_FIX=1                                                                                                        
  int8 STATUS_GBAS_FIX=2                                                                                                        
  uint16 SERVICE_GPS=1                                                                                                          
  uint16 SERVICE_GLONASS=2                                                                                                      
  uint16 SERVICE_COMPASS=4                                                                                                      
  uint16 SERVICE_GALILEO=8                                                                                                      
  int8 status                                                                                                                   
  uint16 service                                                                                                                
float64 latitude                                                                                                                
float64 longitude                                                                                                               
float64 altitude                                                                                                                
float64[9] position_covariance                                                                                                  
uint8 position_covariance_type
"""
class WayPoint():
    def __init__(self, latitude, longitude, altitude):
        self.latitude = latitude
        self.longitude = longitude
        self.altitude = altitude
        
    def print_data(self):
        return "["+str(self.latitude)+","+str(self.longitude)+","+str(self.altitude)+"]"


class GpsClass(object):
    def __init__(self):
        self._sub = rospy.Subscriber('/fix', NavSatFix, self.callback)
        self.latitude = 0.0
        self.longitude = 0.0
        self.altitude = 0.0
        self.testwaypoint = WayPoint(49.900090,8.899960,0.000000)
    
    def remove_noise(self,latitude,longitude,altitude):
        """
        The GPS have Noise, so the always alscilation digits are removed
        """
        return round(latitude,5),round(longitude,5),round(altitude,1)
    
    def callback(self,msg):
        self.latitude, self.longitude, self.altitude = self.remove_noise(msg.latitude, msg.longitude, msg.altitude)
        #rospy.loginfo('Origin [latitude,longitude,altitude]=[%f,%f,%f]', self.latitude, self.longitude, self.altitude)
        #rospy.loginfo('Waypoint [latitude,longitude,altitude]=[%f,%f,%f]', self.testwaypoint.latitude, self.testwaypoint.longitude, self.testwaypoint.altitude)
        #rospy.loginfo('Distance to WayPointTest=%f', self.distance_from_waypoint(self.testwaypoint))
    
    def distance_from_waypoint(self,waypoint):
        """
        It's calculated only with longitude and latitude; altitude oscillates too much for a coherent reading, and 
        in this application, there is no reason to worry about it.
        """
        origin = (self.latitude, self.longitude)
        waypoint = (waypoint.latitude, waypoint.longitude)
        return vincenty(origin, waypoint).meters
    
    def get_current_gps_pos(self):
        """
        Returns newest GPS current position in a Waypoint Format
        """
        return WayPoint(self.latitude, self.longitude, self.altitude)
        

if __name__ == '__main__':
  rospy.init_node('gps_topic_subscriber')
  GpsClass()
  rospy.spin()


ModuleNotFoundError: No module named 'rospy'

### Notes about the GPS Class

<ol>
    <li>**distance_from_waypoint**: This function calculates the distance between the current GPS position and a given GPS Waypoint. This distance is calculated through Vincenty's formula. For more info: https://en.wikipedia.org/wiki/Vincenty's_formulae. This formula calculates the distance between two points in a spheroid, which gives a more precise distance than the classical euclidian distance.</li>
    <li>**remove_noise**: This function crops the GPS data to the digits where the noise stops. Because of how GPS works, there is always noise, and in this simulation, it's simulated to be like this also.</li>
    <li>**testwaypoint**: This waypoint is a point that is useful for testing.</li>
    <li>At what rate is the GPS data published in the topic /fix ? Execute the following command to learn at what rate the values will be updated inside the class:<br><br>
    **<i>rostopic hz /fix**</i><br><br>
    The values will be around:<br>
    average rate: 10.000<br>
    </li>
</ol>

### Move To WayPoint GPS Action Server

This action will wait for a client to send a waypoint number. Once received, it will constantly calculate the distance to that waypoint, and return the distance in meters, in integer format, as feedback. When the distance gets lower than the minimum, it will return the final distance as the result.

In [2]:
#! /usr/bin/env python
import rospy
import actionlib
from gps_reader import GpsClass, WayPoint
from actionlib.msg import TestFeedback, TestResult, TestAction
"""
### Test.action ###
int32 goal                                                                                                                      
---                                                                                                                             
int32 result                                                                                                                    
---                                                                                                                             
int32 feedback # This will give the distance from current pos to waypoint
"""

class MoveToGpsWayPointAServerClass(object):

  def __init__(self):
    # creates the action server
    rospy.init_node('move_to_gps_waypoint_aserver_node')
    # create messages that are used to publish feedback/result
    self._feedback = TestFeedback()
    self._result   = TestResult()
    # init some internal constants
    self._distance_error = 3.0 # A 3 metre error
    self._action_rate = 10.0
    waypoint1 = WayPoint(49.900090,8.899960,0.000000)
    self._waypoint_dict = {1:waypoint1}
    self._gps_class = GpsClass()
    self._as = actionlib.SimpleActionServer("/move_to_gps_waypoint_aserver", TestAction, self.goal_callback, False)
    self._as.start()
    rospy.loginfo('move_to_gps_waypoint_aserver initialised')

    
  def goal_callback(self, goal):
    # this callback is called when the action server is called.
    # this is the function that reads the GPS current position and publishes the appropriate Movement Command
    
    # helper variables
    r = rospy.Rate(self._action_rate)
    success = True
    
    # TODO: Fetch Data from topic GPS
    
    waypoint_gps_pos = self._waypoint_dict.get(goal.goal)
    if waypoint_gps_pos != None:
        
        # Distance from current pos to the given waypoint
        distance = self._gps_class.distance_from_waypoint(waypoint_gps_pos)
        self._feedback.feedback = int(distance)
        self._as.publish_feedback(self._feedback)
        
        start_gps_pos = self._gps_class.get_current_gps_pos()
        # publish info to the console for the user
        rospy.loginfo("#### move_to_gps_waypoint_as:\nStart GPS Position="+ str(start_gps_pos.print_data()) + "\nWayPoint = "+str(waypoint_gps_pos.print_data()) + "\nDistance ="+str(distance)+"\n###")
        
        # starts Moving To the WayPoint
        while int(distance) >= int(self._distance_error):
          rospy.loginfo('Distance From WayPoint:'+str(distance))
          # check that preempt (cancellation) has not been requested by the action client
          if self._as.is_preempt_requested():
            rospy.loginfo('The WayPoint has been cancelled/preempted')
            # the following line, sets the client in preempted state (goal cancelled)
            self._as.set_preempted()
            success = False
            self._result.result = int(success)
            break
          
          # builds the next feedback msg to be sent
          self._feedback.feedback = int(distance)
          # publish the feedback
          self._as.publish_feedback(self._feedback)
          # the sequence is computed at 1 Hz frequency
          r.sleep()
          
          # Update Distance
          distance = self._gps_class.distance_from_waypoint(waypoint_gps_pos)
    
    else:
        success = False
    # at this point, either the goal has been achieved (success==true)
    # or the client preempted the goal (success==false)
    # If success, then we publish the final result
    # If not success, we do not publish anything in the result
    if success:
        rospy.loginfo("### move_to_gps_waypoint_as:\nSucceeded to move from GPS Position="+ str(start_gps_pos.print_data()) + "\nWayPoint = "+str(waypoint_gps_pos.print_data())+"\n###")
        rospy.loginfo('Distance From WayPoint:'+str(distance)+",feedbackdistance="+str(self._feedback.feedback))
    else:
        rospy.loginfo("move_to_gps_waypoint_as: Failed")
    
    self._result.result = int(success)
    self._as.set_succeeded(self._result)
      
if __name__ == '__main__':
  
  MoveToGpsWayPointAServerClass()
  rospy.spin()

ModuleNotFoundError: No module named 'rospy'

### Notes about MoveToGpsWayPointAServerClass

* **The Action messages used**: the messages from the ROS package actionlib are used. The reasons for this are just to avoid creating custom action messages and simplifying the example. This action message has the following structure:<br>

In [3]:
### Test.action ###
int32 goal
---
int32 result
---
int32 feedback # This will give the distance from the current pos to the waypoint

SyntaxError: invalid syntax (<ipython-input-3-babb21475582>, line 2)

The same information can be retrieved with the following commands:

In [ ]:
roscd actionlib/action;
cat Test.action

* The **distance_error** is set to 3.0 meters. If it gets lower than 3, then the action server will consider the action ended and will return the result.


* The **waypoint_dict**: This dictionary carries the waypoints available. In this case, only one has been implemented. If the client asks for a Waypoint number other than 1, the action will end with an error.

### Move To WayPoint GPS Action Client

Here is an example of how to ask the GPS Action Server to perform an action. The client only has to send an integer that represents the number of the Waypoint to be reached as goal.

In [4]:
#! /usr/bin/env python

import rospy
import time
import actionlib
from actionlib.msg import TestAction, TestGoal, TestResult, TestFeedback
"""
### Test.action ###
int32 goal                                                                                                                      
---                                                                                                                             
int32 result                                                                                                                    
---                                                                                                                             
int32 feedback # This will give the distance from the current pos to the waypoint
"""
"""
class SimpleGoalState:
    PENDING = 0
    ACTIVE = 1
    DONE = 2
    WARN = 3
    ERROR = 4

"""
# We create some constants with the corresponding values from the SimpleGoalState class
PENDING = 0
ACTIVE = 1
DONE = 2
WARN = 3
ERROR = 4

# definition of the feedback callback. This will be called when feedback
# is received from the action server
# it just prints a message indicating a new message has been received
def feedback_callback(feedback):
    rospy.loginfo('Feedback Distnace = '+str(feedback.feedback))

# initializes the action client node
rospy.init_node('move_to_gps_waypoint_aclient_node')

action_server_name = '/move_to_gps_waypoint_aserver'
client = actionlib.SimpleActionClient(action_server_name, TestAction)

# waits until the action server is up and running
rospy.loginfo('Waiting for action Server '+action_server_name)
client.wait_for_server()
rospy.loginfo('Action Server Found...'+action_server_name)

# creates a goal to send to the action server
goal = TestGoal()
goal.goal = 1 # indicates, move to waypoint number 1

client.send_goal(goal, feedback_cb=feedback_callback)


# You can access the SimpleAction Variable "simple_state," that will be 1, if active; and 2 when finished.
#It's a variable, so you'd better use a function like get_state.
#state = client.simple_state
# state_result will give the FINAL STATE. Will be 1 when Active, and 2 if NO ERROR, 3 If Any Warning, and 3 if ERROR
state_result = client.get_state()

rate = rospy.Rate(10)

rospy.loginfo("state_result: "+str(state_result))

while state_result < DONE:
    #rospy.loginfo("Doing Stuff while waiting for the Server to give a result....Nothing for the moment")
    rate.sleep()
    state_result = client.get_state()
    #rospy.loginfo("state_result: "+str(state_result))
    
rospy.loginfo("[Result] State: "+str(state_result))
if state_result == ERROR:
    rospy.logerr("Something went wrong in the Server Side")
if state_result == WARN:
    rospy.logwarn("There is a warning in the Server Side")


ModuleNotFoundError: No module named 'rospy'

<p style="background:green;color:white;">END Solution Exercise U2.1</p>

## Congratulations! You now know how to navigate with GPS Data.

In the next unit, you will learn to implement an obstacle detection system using the laser data, and also what the DeadMansSwitch system is in cars.